In [ ]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor

In [ ]:
from sklearn.metrics import make_scorer

# function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def Root_MSLE(truth, prediction):
  # prediction = prediction.clip(min=0)
  # prediction = prediction.clip(max=15)

  msle = np.mean((np.log(truth + 1) - np.log(prediction + 1)) ** 2)
  return np.sqrt(msle)

rmsle = make_scorer(Root_MSLE, greater_is_better=False)

# Baseline model

In [ ]:
df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/heritage health prize/final_data.csv")

data = df[df['Year']=='Y1'].copy()
data_test = df[df['Year']=='Y2'].copy()

In [ ]:
# len(data.index)

In [ ]:
# exclude some definition features from our training set
features_to_drop = ('MemberID', 'DaysInHospital', 'Year', 'trainset', 'in_hospital')
features = [i for i in data.columns if i not in features_to_drop]

X_train = data[features]
y_train = data['DaysInHospital']

X_test = data_test[features]
y_test = data_test['DaysInHospital']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
# run model without cross validation
def model_evaluation(model, train_feature=X_train, train_target=y_train, test_feature=X_test, test_target=y_test):
  model.fit(train_feature, train_target)
  prediction = model.predict(test_feature)
  prediction = prediction.clip(min=0)
  prediction = prediction.clip(max=15)
  return Root_MSLE(test_target, prediction)

# declaring models
xgb_reg = XGBRegressor(objective='reg:squarederror', verbosity=1)
lin_reg = LinearRegression()
ridge_reg = Ridge()
lasso_reg = Lasso()
rf_reg = RandomForestRegressor(max_depth=10)
lin_sv_reg = LinearSVR(max_iter=1000, random_state=42)
sgd_reg = SGDRegressor(early_stopping=True, random_state=42)

print("Model Performance with Default Hyper Parameter Settings")
print(f"RMSLE from Gradient Boosting Machines: {model_evaluation(xgb_reg):.4f}")
print(f"RMSLE from Linear Regression: {model_evaluation(lin_reg):.4f}")
print(f"RMSLE from Ridge Regression: {model_evaluation(ridge_reg):.4f}")
print(f"RMSLE from Lasso Regression: {model_evaluation(lasso_reg):.4f}")
print(f"RMSLE from Random Forest Regression: {model_evaluation(rf_reg):.4f}")
print(f"RMSLE from Linear Support Vector Regression: {model_evaluation(lin_sv_reg):.4f}")
print(f"RMSLE from SGD Regression: {model_evaluation(sgd_reg):.4f}")

Model Performance with Default Hyper Parameter Settings
RMSLE from Gradient Boosting Machines: 0.4935
RMSLE from Linear Regression: 1.6554
RMSLE from Ridge Regression: 0.4999
RMSLE from Lasso Regression: 0.5165
RMSLE from Random Forest Regression: 0.4956


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


RMSLE from Linear Support Vector Regression: 0.5038
RMSLE from SGD Regression: 0.5344


$\rightarrow$ Select *GBM, Ridge Regression, Random Forest Regression* for our ensemble

## Hyper parameter tuning

$\rightarrow$ Use `RandomSearchCV` to find the top 20 performing set of hyper parameters

In [ ]:
# xgb_reg = XGBRegressor(objective='reg:squarederror', subsample=0.3
#                       , colsample_bylevel=.3, colsample_bynode=.3, colsample_bytree=0.3
#                       , learning_rate=.01, max_depth=10, reg_lambda=40, n_estimators=500)

# model_evaluation(xgb_reg)

0.49058832266132385

Due to computational limitation, `RandomSearchCV` or `GridSearchCV` are not performing in a reasonable amount of time. Therefore, I opt to go with another approach.

Take 20 random combinations of the hyper parameters and build models based on these combinations, which makes 40 total models for *XGBoost* and *Random Forest*. Next, choose the top 20 best models from these for the final ensemble.

In [ ]:
# make parameter combinations for XGBoost
xgb_params_keys = [
                   'objective', 'subsample', 'colsample_bylevel', 'colsample_bynode',
                   'colsample_bytree', 'learning_rate', 'max_depth', 'reg_lambda', 'n_estimators'
]

xgb_params_values = [
    ['reg:squarederror']
    , np.arange(0.1, 1.1, .1)
    , np.arange(0.1, 1.1, .1)
    , np.arange(0.1, 1.1, .1)
    , np.arange(0.1, 1.1, .1)
    , [0.0001, 0.001, 0.01, 0.1]
    , [3, 5, 7, 10]
    , [40]
    , [50, 100, 150, 200, 500, 1000]
]

xgb_params_grid = {}
for i in range(20):
  new_random_xgb_params_values = []
  for j in range(len(xgb_params_values)):
    new_random_xgb_params_values.append(random.choice(xgb_params_values[j]))
  
  xgb_params_grid['xgb set {}'.format(i)] = dict(zip(xgb_params_keys, new_random_xgb_params_values))

# save the results in a dictionary
xgb_model_eval = {}
xgb_ypred = {}
print("Hyper parameter tuning for Boosting Machines:\n")
for k in xgb_params_grid.keys():
  xgb_reg = XGBRegressor(**xgb_params_grid[k])
  xgb_reg.fit(X_train, y_train)
  y_pred = xgb_reg.predict(X_test)

  score = Root_MSLE(y_test, y_pred)
  xgb_ypred[k] = y_pred

  xgb_model_eval[k] = score
  
  print('RMSLE for {} is {}'.format(k, score))

  # break

Hyper parameter tuning for Boosting Machines:

RMSLE for xgb set 0 is 0.49250420223013996
RMSLE for xgb set 1 is 0.5062484856638836
RMSLE for xgb set 2 is 0.5168341324780346
RMSLE for xgb set 3 is 0.49189251000022227
RMSLE for xgb set 4 is 0.5254492703423684
RMSLE for xgb set 5 is 0.5164247713927206
RMSLE for xgb set 6 is 0.4949305951880867
RMSLE for xgb set 7 is 0.5255140215716714
RMSLE for xgb set 8 is 0.5228427809638113
RMSLE for xgb set 9 is 0.49378651429140424
RMSLE for xgb set 10 is 0.48744855020788147
RMSLE for xgb set 11 is 0.5187923410793401
RMSLE for xgb set 12 is 0.5003189000750741
RMSLE for xgb set 13 is 0.49167472287391617
RMSLE for xgb set 14 is 0.5186495324230135
RMSLE for xgb set 15 is 0.494113976854677
RMSLE for xgb set 16 is 0.518746516992188
RMSLE for xgb set 17 is 0.4918391989874137
RMSLE for xgb set 18 is 0.48954202011720727
RMSLE for xgb set 19 is 0.4907883346903613


In [ ]:
# xgb_ypred
# len(X_test.index)
# X_test[0]

In [ ]:
# make combinations for Random Forest
rf_params_keys = ['n_estimators', 'max_depth', 'max_samples', 'max_features']

rf_params_values = [
  [50, 100, 150, 200]
  , [3, 5, 7, 10]
  , [.1, .2, .3, .4, .5, .6, .7, .8, .9]
  , [.1, .2, .3, .4, .5, .6, .7, .8, .9]
]

rf_params_grid = {}
for i in range(20):
  new_random_rf_params_values = []
  for j in range(len(rf_params_values)):
    new_random_rf_params_values.append(random.choice(rf_params_values[j]))
  
  rf_params_grid['rf set {}'.format(i)] = dict(zip(rf_params_keys, new_random_rf_params_values))

# save the results in a dictionary
rf_model_eval = {}
rf_ypred = {}
print("Hyper parameter tuning for Random Forest:\n")
for k in rf_params_grid.keys():
  rf_reg = RandomForestRegressor(**rf_params_grid[k])
  rf_reg.fit(X_train, y_train)
  y_pred = rf_reg.predict(X_test)

  score = Root_MSLE(y_test, y_pred)

  rf_ypred[k] = y_pred
  rf_model_eval[k] = score
  
  print('RMSLE for {} is {}'.format(k, score))

Hyper parameter tuning for Random Forest:

RMSLE for rf set 0 is 0.4943691431306355
RMSLE for rf set 1 is 0.49372439164143955
RMSLE for rf set 2 is 0.49421575724347877
RMSLE for rf set 3 is 0.4980308002118516
RMSLE for rf set 4 is 0.49322779711552417
RMSLE for rf set 5 is 0.4946372965949316
RMSLE for rf set 6 is 0.4962381642828274
RMSLE for rf set 7 is 0.49602748637360294
RMSLE for rf set 8 is 0.49456739295506796
RMSLE for rf set 9 is 0.4951205473589937
RMSLE for rf set 10 is 0.49577354331789136
RMSLE for rf set 11 is 0.4941598883175722
RMSLE for rf set 12 is 0.4981728092124775
RMSLE for rf set 13 is 0.4942813551012194
RMSLE for rf set 14 is 0.4945190420479539
RMSLE for rf set 15 is 0.49482953773481725
RMSLE for rf set 16 is 0.496654209084486
RMSLE for rf set 17 is 0.4964479646417893
RMSLE for rf set 18 is 0.4938427866671478
RMSLE for rf set 19 is 0.49553590440835027


In [ ]:
rf_model_eval.update(xgb_model_eval)

model_eval_df = pd.DataFrame.from_dict(rf_model_eval, orient='index', columns=['rmsle_score']).sort_values(by='rmsle_score')

selected_models = model_eval_df.head(20)

# Ensemble & Cross validation

In [ ]:
ridge_ypred = ridge_reg.fit(X_train, y_train).predict(X_test)

xgb_ypred.update(rf_ypred)
xgb_ypred['ridge_ypred'] = ridge_ypred

In [ ]:
ensemble_features = selected_models.index.to_list() + ['ridge_ypred']

ensemble_X = pd.DataFrame()
for label in ensemble_features:
  ensemble_X.loc[:, label] = xgb_ypred[label]

ensemble_y = y_test.copy()

In [ ]:
ensemble_X.head()

,xgb set 10,xgb set 18,xgb set 19,xgb set 13,xgb set 17,xgb set 3,xgb set 0,rf set 4,rf set 1,xgb set 9,rf set 18,xgb set 15,rf set 11,rf set 2,rf set 13,rf set 0,rf set 14,rf set 8,rf set 5,rf set 15,ridge_ypred
0,0.441978,0.487561,0.558305,0.430259,0.477640,0.526588,0.542039,0.353051,0.385026,0.350562,0.480955,0.609753,0.352823,0.307695,0.280881,0.262856,0.413562,0.359268,0.354161,0.261325,0.557741
1,0.312300,0.298846,0.362921,0.419430,0.446987,0.331339,0.285099,0.498372,0.473775,0.389514,0.432565,0.317982,0.481623,0.537192,0.518726,0.502789,0.389244,0.459487,0.506844,0.512859,0.053142
2,0.485472,0.194081,0.420051,0.336403,0.429280,0.165904,0.246369,0.182874,0.236698,0.517669,0.280777,0.185024,0.202579,0.217575,0.206179,0.210051,0.201390,0.203680,0.191286,0.213459,0.365717
3,0.183506,0.120559,0.142099,0.292311,0.321424,0.137351,0.125706,0.168750,0.187367,0.138082,0.177981,0.118286,0.178555,0.200256,0.200643,0.203265,0.160761,0.182006,0.177586,0.202147,0.206072
4,0.064159,0.114436,0.090029,0.289856,0.272544,0.115143,0.118966,0.168750,0.179927,0.076943,0.141666,0.121111,0.177883,0.199699,0.200643,0.203265,0.147952,0.179239,0.177586,0.202147,0.085561


We use *Linear Regression*, without the intercept, to get the weights for these models

In [ ]:
ens_lin_reg = LinearRegression(fit_intercept=False, normalize=True)

In [ ]:
# prepare the cross-validation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

# evaluate model
scores = cross_val_score(ens_lin_reg, ensemble_X, ensemble_y, scoring=rmsle, cv=cv, n_jobs=-1)

# report performance
print('RMSLE on train set: %.3f (%.3f)' % (-np.mean(scores), np.std(scores)))

RMSLE on train set: 0.485 (0.006)


In [ ]:
ens_lin_reg.fit(ensemble_X, ensemble_y)

ens_ypred = np.dot(ensemble_X.values, ens_lin_reg.coef_)

print("RMSLE on the test set: ", Root_MSLE(y_test, ens_ypred))

RMSLE on the test set:  0.484448023276567


In [ ]:
# selected_models

,rmsle_score
xgb set 10,0.487449
xgb set 18,0.489542
xgb set 19,0.490788
xgb set 13,0.491675
xgb set 17,0.491839
xgb set 3,0.491893
xgb set 0,0.492504
rf set 4,0.493228
rf set 1,0.493724
xgb set 9,0.493787
